In [214]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os
import sys
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
sys.path.append('/Users/mintflavor/내 드라이브/dacon/Study/edgedriver_mac64')

In [215]:
#함수 선언
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

In [236]:
def save():
    if not os.path.exists('output1.csv'):
        data.to_csv('output1.csv', encoding='utf-8-sig', mode='w', index=False)
    else:
        data.to_csv('output1.csv',encoding='utf-8-sig', mode='a', header=False, index=False)

In [232]:
searchItems = {'macbook air m1 256 8 실버' : 'https://search.shopping.naver.com/catalog/25039820524?query=%EB%A7%A5%EB%B6%81&NaPm=ct%3Dkyzer2fs%7Cci%3Dd029492d7a26883363de2b565d9843ff73bab8a2%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D1875370e23e09feda6bb1a895bb905a727d8edef',
               'macbook air m1 256 8 스페이스 그레이' : 'https://search.shopping.naver.com/catalog/26101847522?query=%EB%A7%A5%EB%B6%81%EC%97%90%EC%96%B4&NaPm=ct%3Dkyzjsvyo%7Cci%3D5f8bb1844314417faeac19c9d083d559dc95a594%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Df34443d7cad0fb9bf5d449267599a5728eb49467',
               'macbook air m1 256 16' : 'https://search.shopping.naver.com/catalog/26380658523?query=%EB%A7%A5%EB%B6%81%EC%97%90%EC%96%B4&NaPm=ct%3Dkyzju3y0%7Cci%3D2fcf65071dce05a2bdd04797b2b84f13ef217693%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D00dee5ab23c6a072325fabecb4fa487c6a57d5ce',
               'macbook air m1 512 16' : 'https://search.shopping.naver.com/catalog/26380764523?query=%EB%A7%A5%EB%B6%81%EC%97%90%EC%96%B4&NaPm=ct%3Dkyzjvf0g%7Cci%3Dda1c51f980973eb80a5fb7fbcb8060ad8c2f1875%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D8ee523be4a7a4ab416cae7009836aff6f60652ed',
              'macbook pro m1 13 256 8 실버' : 'https://search.shopping.naver.com/catalog/25040992522?query=%EB%A7%A5%EB%B6%81%20%ED%94%84%EB%A1%9C&NaPm=ct%3Dkyzjwcyo%7Cci%3D4e053b1216cd7f62715d53d2afaf9de8764a6f82%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D52f9bab71a9fe6b6dd311d94014032b8f3251680',
               'macbook pro m1 13 512 16' : 'https://search.shopping.naver.com/catalog/26380935522?query=%EB%A7%A5%EB%B6%81%20%ED%94%84%EB%A1%9C&NaPm=ct%3Dkz0v1pwg%7Cci%3D906cc8d653bef28a9a56870d98ec8ec60624475f%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D0cf89a76abaf4f8db5e32e5dd8a336ae5c1a74e6',
              'macbook pro m1 13 256 8 스페이스 그레이' : 'https://search.shopping.naver.com/catalog/25040163523?query=%EB%A7%A5%EB%B6%81%ED%94%84%EB%A1%9C&NaPm=ct%3Dkyzimrrc%7Cci%3D51099284d3cb7a491a9194d19c43df0c80d29a56%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dc7721598eaae05b84b36467b91687a79897170c3'}

In [218]:
#xpath
                    #/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[1]/a/strong
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div/ul"

In [233]:
data = pd.DataFrame(columns=['type','category','review','star'])

In [234]:
header = {'User-Agent': ''}
for key in tqdm(list(searchItems.keys())):
    #브라우저 오픈
    d = webdriver.Edge() # webdriver = Edge, 웹드라이버는 본인 환경에 맞게 알잘딱깔센
    d.implicitly_wait(3)
    d.get(searchItems[key])
    req = requests.get(searchItems[key])
    html = req.text 
    soup = BeautifulSoup(html, "html.parser")
    
    #쇼핑몰 리뷰 보기
    element = d.find_element_by_xpath(shoppingmall_review)
    d.execute_script("arguments[0].click();", element)
    tabLength = len(element.text.split('\n'))
    tabIndex = tabLength+1
    #print(tabIndex, tabLength)
    
    d.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    sleep(2)
    
    #"전체" 리뷰 가져오기
    name_=key 
    category_="전체"
    reviews=[]
    stars=[]
    cnt=1   #리뷰index
    page=1
    while True:
        #print("페이지", page ,"\n") 
        sleep(2)
        for j in range(1, 21): #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
            try:
                star=d.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[{tabIndex}]/ul/li['+str(j)+']/div[1]/span[1]').text
                stars.append(star)
                review=d.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[{tabIndex}]/ul/li['+str(j)+']/div[2]/div[1]/p').text                
                review.replace('\n', ' ')
                reviews.append(review)
                if j%2==0: #화면에 2개씩 보이도록 스크롤
                    ELEMENT = d.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[{tabIndex}]/ul/li['+str(j)+']/div[2]/div[1]/p')
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                #print(cnt, review ,star, "\n")
                cnt+=1
            except:
                break

        #sleep(2)

        if page<11:#page10까지 적용
            try: #리뷰의 마지막 페이지에서 error발생
                page+=1
                next_page=d.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[{tabIndex}]/div[3]/a['+str(page)+']').click() 
                #sleep(2)
            except: #리뷰의 마지막 페이지에서 process 종료 
                break

        else :
            try: #page11부터
                if page%10==0:
                    next_page=d.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[{tabIndex}]/div[3]/a[12]').click()
                else:
                    next_page=d.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[{tabIndex}]/div[3]/a['+str(page%10+2)+']').click()
                page+=1
                #sleep(2)
            except:
                break

    df1=add_dataframe(name_,category_,reviews,stars,cnt)
    data = pd.concat([data, df1], axis=0)
    d.quit()
data.reset_index(drop=True, inplace=True)
save()

100%|█████████████████████████████████████████████| 7/7 [06:48<00:00, 58.32s/it]


In [226]:
d.quit()

In [228]:
data[data['type'] == 'macbook pro m1 13 256 8 실버']

,type,category,review,star
1493,macbook pro m1 13 256 8 실버,전체,"먼저 제품은 만족합니다.\n다만, 실버 색상은 기간이 오래걸린다고 하여, 제 아내의...",평점2
1494,macbook pro m1 13 256 8 실버,전체,음 16GB 살껄 하는 생각이 드는데\n만약 조금이라도 고민 되시는 분들은 그냥 1...,평점5
1495,macbook pro m1 13 256 8 실버,전체,2015년형 맥북 쓰다가 침수로 못살려낼줄 알고 구매했는데\n침수되었던 맥북 회생(...,평점4
1496,macbook pro m1 13 256 8 실버,전체,공식스토어라고 해서 여기서 구입했는데. 배송 문제로 속상해서 인지 도착한 맥북이 하...,평점2
1497,macbook pro m1 13 256 8 실버,전체,배송은 1-2주 걸린다 했었는데 3주 넘게 걸렸어요. 지연되거나 하면 미리 연락 주...,평점5
...,...,...,...,...
1626,macbook pro m1 13 256 8 실버,전체,조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조...,평점5
1627,macbook pro m1 13 256 8 실버,전체,배송빠릅니다 배송빠릅니다 배송빠릅니다,평점4
1628,macbook pro m1 13 256 8 실버,전체,감사합니다 감사합니다 감사합니다,평점5
1629,macbook pro m1 13 256 8 실버,전체,조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조아요조...,평점5


In [235]:
data

,type,category,review,star
0,macbook air m1 256 8 실버,전체,이번 M1은 정말 물건입니다.\n기존 2019 맥북프로 고급형 16인치 사용하고 있...,평점5
1,macbook air m1 256 8 실버,전체,장점이 많은 M1제품. 2017년형 맥북프로 13인치 사용중이였는데 훨씬 낫네요. ...,평점5
2,macbook air m1 256 8 실버,전체,한 4일 써보고 리뷰 남깁니다.\n진짜 하루라도 빨리 살 걸 후회합니다.\n팬리스로...,평점5
3,macbook air m1 256 8 실버,전체,솔직히 첨엔 이쁜 거 '디자인' 보고 사고 싶단 생각을 하기 시작했는데요ㅎㅎㅎ(애플...,평점5
4,macbook air m1 256 8 실버,전체,배송이 조금 늦은감이 있었지만 상품은 아주 기다리고 기대했던 만큼 최고네요아이폰 아...,평점5
...,...,...,...,...
1899,macbook pro m1 13 256 8 스페이스 그레이,전체,배송 총알이네여 ㅎㅎ 판매자님 감사합니다 포장꼼꼼히와서 좋아요 제품 이상없이 잘왔습...,평점5
1900,macbook pro m1 13 256 8 스페이스 그레이,전체,6322282993827282,평점5
1901,macbook pro m1 13 256 8 스페이스 그레이,전체,맥북은 맥북이다 엠1은 역대급 맥북이다,평점5
1902,macbook pro m1 13 256 8 스페이스 그레이,전체,감사합니다. 잘받았습니다. 감사합니다.,평점4


In [237]:
save()